In [12]:
import pandas as pd
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')

In [22]:
df = pd.read_csv('BTC_5min_10AUG.csv')

In [23]:
def sma(data, window):
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

In [24]:
symbols = ['BTC']

In [25]:
nstd = 3

for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'],20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)
    df[f'{symbol}_rsi'] = ta.RSI(df[f'{symbol}-USD_Close'])


In [26]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier, symbols):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_multiplier # VIP level 0, paying fees with BNB
        self.symbols = symbols
        
        self.bottoms = {}
        self.reset_bottoms() 
        
        self.tops = {}
        self.reset_tops()
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append([self.balance_unit, time, sell_price])
        self.balance_unit = 'USDT'
        
    def reset_bottoms(self):
        for symbol in self.symbols:
            self.bottoms[symbol] = 'none'
        
    def reset_tops(self):
        for symbol in self.symbols:
            self.tops[symbol] = 'none'
        

In [27]:
'''
innitially when price is within channel = none
when it crossed first bottom = hit
when it crosses above bottom after hit = released
'''

'\ninnitially when price is within channel = none\nwhen it crossed first bottom = hit\nwhen it crosses above bottom after hit = released\n'

In [28]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=0.99925, symbols=symbols)

for i in range(len(df)):  
    #means looking for something to buy
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if env.bottoms[symbol] == 'hit' and df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                env.bottoms[symbol] = 'released'
            
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] < 20: #buy signal
                if env.bottoms[symbol] == 'released':
                    env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                    env.reset_bottoms()
                    break
                else:
                    env.bottoms[symbol] = 'hit'
    
    #means looking to sell
    if env.balance_unit != 'USDT':
        if env.tops[env.balance_unit] == 'hit' and df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]:
            env.tops[env.balance_unit] = 'released'
        
        if df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] > 80: #sell signal
            if env.tops[env.balance_unit] == 'released':
                env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])
                env.reset_tops()
            else:
                env.tops[env.balance_unit] = 'hit'

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])

In [29]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 17
num sells: 17
ending balance: 80.21181363243893 USDT


In [20]:
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    print('\n')

['BTC', '2022-04-11 19:55:00', 41679.85598177473]
['BTC', '2022-04-19 03:10:00', 39800.09052714683]
P/L = -4.5100094766398975


['BTC', '2022-04-24 09:30:00', 39712.096061841]
['BTC', '2022-04-26 04:50:00', 40033.523797912625]
P/L = 0.8093950406724559


['BTC', '2022-04-29 20:45:00', 38974.42302031913]
['BTC', '2022-05-04 16:05:00', 38182.85574936885]
P/L = -2.030991634020096


['BTC', '2022-05-05 23:55:00', 39026.92706598513]
['BTC', '2022-05-22 19:30:00', 29684.3739913947]
P/L = -23.938736090582857


['BTC', '2022-05-24 05:20:00', 29360.50061456294]
['BTC', '2022-05-30 08:35:00', 29472.70756165124]
P/L = 0.38216973396101306


['BTC', '2022-06-07 10:25:00', 30864.224501865352]
['BTC', '2022-06-08 05:35:00', 30712.257227913033]
P/L = -0.4923735373397631


['BTC', '2022-06-08 13:30:00', 30223.323719801294]
['BTC', '2022-06-20 04:55:00', 19829.71009910894]
P/L = -34.38937992740624


['BTC', '2022-06-25 22:30:00', 21094.126000241362]
['BTC', '2022-06-28 19:05:00', 21196.918177763997]
P/L 